# Tutorial6: Memory-efficient aggregations

## Outline

- Representation of sparse matrices
- The `SparseTensor` class
- Use of `SparseTensor` in PyG

Official resources:
* [PyG documentation](https://pytorch-geometric.readthedocs.io/en/latest/notes/sparse_tensor.html)

### Imports

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 16})

## Representation of sparse matrices

See [Scipy sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)

### A sparse matrix

In [ ]:
import numpy as np

n = 5
row = [2, 1, 4, 0, 2, 4]
col = [2, 2, 1, 3, 1, 0]
data = [1, 2, 3, 4, 5, 6]
A = np.zeros((n, n))
A[row, col] = data
A

array([[0., 0., 0., 4., 0.],
       [0., 0., 2., 0., 0.],
       [0., 5., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       [6., 3., 0., 0., 0.]])

### `coo` format (coordinate format)

**Good:** Easy construction and conversion to different formats

**Bad:** Arithmetic operations, slicing

In [ ]:
from scipy.sparse import coo_matrix

A_coo = coo_matrix((data, (row, col)), shape=(n, n))
A_coo

<COOrdinate sparse matrix of dtype 'int64'
	with 6 stored elements and shape (5, 5)>

In [4]:
A_coo.toarray()

array([[0, 0, 0, 4, 0],
       [0, 0, 2, 0, 0],
       [0, 5, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [6, 3, 0, 0, 0]])

In [5]:
A_coo.todense()

matrix([[0, 0, 0, 4, 0],
        [0, 0, 2, 0, 0],
        [0, 5, 1, 0, 0],
        [0, 0, 0, 0, 0],
        [6, 3, 0, 0, 0]])

In [6]:
A_coo_from_dense = coo_matrix(A)
A_coo_from_dense.toarray()

array([[0., 0., 0., 4., 0.],
       [0., 0., 2., 0., 0.],
       [0., 5., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       [6., 3., 0., 0., 0.]])

### `csr` format (compressed sparse row format)

**Good:** Fast entry-wise arithmetic operations, efficient row slicing, fast matrix-vector products

**Bad:** Slow column slicing, expensive changes of the sparsity pattern

In [ ]:
from scipy.sparse import csr_matrix

A

array([[0., 0., 0., 4., 0.],
       [0., 0., 2., 0., 0.],
       [0., 5., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       [6., 3., 0., 0., 0.]])

In [ ]:
# Column indices, sorted according to the row
indices = [
    3,  # indices of the non zero column in the row 0
    2,  # indices of the non zero column in the row 1
    1,
    2,  # indices of the non zero column in the row 2
    # indices of the non zero column in the row 3
    0,
    1,
]   # indices of the non zero column in the row 4

In [ ]:
# Data sorted in the same order
data = [4, 2, 5, 1, 6, 3]

In [ ]:
# Position of the column indices for each row
indptr = [
    0,  # Always start from 0
    1,  # Row 0: the column idx are in position 0:1
    2,  # Row 1: the column idx are in position 1:2
    4,  # Row 2: the column idx are in position 2:4
    4,  # Row 3: the column idx are in position 4:4 (no values)
    6,
]       # Row 4: the column idx are in position 4:6

In [11]:
A_csr = csr_matrix((data, indices, indptr), shape=(n, n))
A_csr

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 6 stored elements and shape (5, 5)>

In [12]:
A_csr.toarray()

array([[0, 0, 0, 4, 0],
       [0, 0, 2, 0, 0],
       [0, 5, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [6, 3, 0, 0, 0]])

### `csc` format (compressed sparse column format)

**Good:** Fast entry-wise arithmetic operations, efficient column slicing, ok matrix-vector products (csr is usually faster)

**Bad:** Slow row slicing, expensive changes of the sparsity pattern

In [ ]:
from scipy.sparse import csc_matrix

A

array([[0., 0., 0., 4., 0.],
       [0., 0., 2., 0., 0.],
       [0., 5., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       [6., 3., 0., 0., 0.]])

In [ ]:
# Row indices, sorted according to the column
indices = [
    4,  # indices of the non zero column in the column 0
    2,
    4,  # indices of the non zero column in the column 1
    1,
    2,  # indices of the non zero column in the column 2
    0,  # indices of the non zero column in the column 4
]       # indices of the non zero column in the column 5

In [ ]:
# Data sorted in the same order
data = [
    6,
    5,
    3,
    2,
    1,
    4,
]

In [ ]:
# Position of the row indices for each column
indptr = [
    0,  # Always start from 0
    1,  # Column 0: the row idx are in position 0:1
    3,  # Column 1: the row idx are in position 1:3
    5,  # Column 2: the row idx are in position 3:5
    6,  # Column 3: the row idx are in position 5:6 (no values)
    6,
]       # Column 4: the row idx are in position 6:6 (no values)

In [17]:
A_csc = csc_matrix((data, indices, indptr), shape=(n, n))
A_csc

<Compressed Sparse Column sparse matrix of dtype 'int64'
	with 6 stored elements and shape (5, 5)>

In [18]:
A_csc.toarray()

array([[0, 0, 0, 4, 0],
       [0, 0, 2, 0, 0],
       [0, 5, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [6, 3, 0, 0, 0]])

## The `SparseTensor` class
* [PyTorch Sparse documentation](https://github.com/rusty1s/pytorch_sparse)

In [ ]:
import torch
from torch_sparse import SparseTensor

In [ ]:
# But col and row need to have dtype torch.long
A_st = SparseTensor(
    row=torch.Tensor(row).to(torch.long),
    col=torch.Tensor(col).to(torch.long),
    value=torch.Tensor(data),
    sparse_sizes=(n, n),
)

In [21]:
A_st

SparseTensor(row=tensor([0, 1, 2, 2, 4, 4]),
             col=tensor([3, 2, 1, 2, 0, 1]),
             val=tensor([2., 5., 1., 6., 4., 3.]),
             size=(5, 5), nnz=6, density=24.00%)

In [22]:
A_st.to_dense()

tensor([[0., 0., 0., 2., 0.],
        [0., 0., 5., 0., 0.],
        [0., 1., 6., 0., 0.],
        [0., 0., 0., 0., 0.],
        [4., 3., 0., 0., 0.]])

In [23]:
A_st_no_val = SparseTensor(
    row=torch.Tensor(row).to(torch.long),
    col=torch.Tensor(col).to(torch.long),
    sparse_sizes=(n, n),
)
A_st_no_val.to_dense()

tensor([[0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.]])

Creation of `SparseTensor`

In [24]:
SparseTensor.from_dense(torch.tensor(A))

SparseTensor(row=tensor([0, 1, 2, 2, 4, 4]),
             col=tensor([3, 2, 1, 2, 0, 1]),
             val=tensor([4., 2., 5., 1., 6., 3.], dtype=torch.float64),
             size=(5, 5), nnz=6, density=24.00%)

In [25]:
SparseTensor.from_scipy(A_coo)

SparseTensor(row=tensor([0, 1, 2, 2, 4, 4]),
             col=tensor([3, 2, 1, 2, 0, 1]),
             val=tensor([4, 2, 5, 1, 6, 3]),
             size=(5, 5), nnz=6, density=24.00%)

Conversion to standard fomats:

In [26]:
row, col, value = A_st.coo()
# rowptr, col, value = A_st.csr()
# colptr, row, value = A_st.csc()
row, col, value

(tensor([0, 1, 2, 2, 4, 4]),
 tensor([3, 2, 1, 2, 0, 1]),
 tensor([2., 5., 1., 6., 4., 3.]))

Basic operations:

In [ ]:
# A_st = A_st[:100, :100]  # Slicing, indexing and masking support
# A_st = A_st.set_diag()   # Add diagonal entries
# A_st_t = A_st.t()        # Transpose
# out = A_st.matmul(x)     # Sparse-dense matrix multiplication
# A_st = A_st.matmul(A_st)  # Sparse-sparse matrix multiplication

## Use of `SparseTensor` in PyG

### Representation of adjacency matrices

In [28]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid('dataset/Planetoid', name='Cora')

Processing...
Done!


In [29]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [30]:
edge_index = dataset.data.edge_index
edge_index

/home/kklepikov/_code_/PytorchGeometricTutorial/.venv/lib/python3.12/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])

In [31]:
num_nodes = dataset[0].x.shape[0]
adj = SparseTensor(
    row=edge_index[0],
    col=edge_index[1],
    sparse_sizes=(num_nodes, num_nodes),
)
adj

SparseTensor(row=tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
             col=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
             size=(2708, 2708), nnz=10556, density=0.14%)

Direct conversion:

In [32]:
import torch_geometric.transforms as T

dataset_st = Planetoid('dataset/Planetoid', name='Cora', transform=T.ToSparseTensor())
dataset_st[0]

Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])

In [33]:
dataset_st[0].adj_t

SparseTensor(row=tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
             col=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
             size=(2708, 2708), nnz=10556, density=0.14%)

Back to `edge_index`:

In [34]:
row, col, edge_attr = dataset_st[0].adj_t.t().coo()
edge_index = torch.stack([row, col], dim=0)
edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

### Usage in the `forward` method

In [35]:
from torch_geometric.nn import GCNConv

conv = GCNConv(dataset.data.x.shape[1], 4)

In [36]:
out1 = conv(dataset.data.x, dataset[0].edge_index)
out1

tensor([[-0.0319, -0.0316,  0.1168,  0.0217],
        [ 0.1020,  0.2406,  0.0250, -0.0422],
        [ 0.1049,  0.1904,  0.1225, -0.0343],
        ...,
        [-0.0712,  0.0105,  0.0312,  0.0816],
        [ 0.0471,  0.0431, -0.0758, -0.0084],
        [-0.0032,  0.0936, -0.0348, -0.0575]], grad_fn=<AddBackward0>)

In [37]:
out2 = conv(dataset[0].x, adj.t())
out2

/home/kklepikov/_code_/PytorchGeometricTutorial/.venv/lib/python3.12/site-packages/torch_sparse/tensor.py:574: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(rowptr, col, value, self.sizes())


tensor([[-0.0319, -0.0316,  0.1168,  0.0217],
        [ 0.1020,  0.2406,  0.0250, -0.0422],
        [ 0.1049,  0.1904,  0.1225, -0.0343],
        ...,
        [-0.0712,  0.0105,  0.0312,  0.0816],
        [ 0.0471,  0.0431, -0.0758, -0.0084],
        [-0.0032,  0.0936, -0.0348, -0.0575]], grad_fn=<AddBackward0>)